In [13]:
import os
from pathlib import Path

import pandas as pd
import xlwings as xw

In [5]:
this_dir = Path(os.getcwd())

In [6]:
parts = []
for file in (this_dir / "sales_data").rglob("*.xls*"):
    print(f'Reading {file.name}')
    part = pd.read_excel(file)
    parts.append(part)

Reading February.xlsx
Reading August.xlsx
Reading April.xlsx
Reading June.xlsx
Reading October.xlsx
Reading September.xlsx
Reading July.xlsx
Reading January.xlsx
Reading May.xlsx
Reading November.xlsx
Reading December.xlsx
Reading March.xlsx
Reading October.xls
Reading September.xls
Reading April.xls
Reading June.xls
Reading January.xls
Reading July.xls
Reading February.xls
Reading December.xls
Reading August.xls
Reading November.xls
Reading March.xls
Reading May.xls


In [7]:
df = pd.concat(parts)

In [8]:
pivot = pd.pivot_table(df,
                       index="transaction_date", columns="store",
                       values="amount", aggfunc="sum")

In [9]:
summary = pivot.resample("M").sum()
summary.index.name = "Month"

In [10]:
summary = summary.loc[:, summary.sum().sort_values().index]

In [11]:
summary.loc[:, "Total"] = summary.sum(axis=1)
summary.loc['Total'] = summary.sum()

In [14]:
template = xw.Book(this_dir /"sales_report_template.xlsx")
sheet = template.sheets["Sheet1"]
sheet["B3"].value = summary
sheet["B3"].expand().columns.autofit()
sheet.charts["Chart 1"].set_source_data(sheet["B3"].expand()[:-1, :-1])
template.save(this_dir / "sales_report_xlwings.xlsx")